In [75]:
import pandas as pd
import missingno as msno 
import lightgbm as lgb

In [76]:
train=pd.read_csv(r'C:\Users\liuchang\Desktop\机器学习-王然老师\ML_dataset-作业数据集\final\train_final.csv')
test=pd.read_csv(r'C:\Users\liuchang\Desktop\机器学习-王然老师\ML_dataset-作业数据集\final\test_final.csv')

In [77]:
(train.isnull().sum()/len(train)).sort_values(ascending=False)

continuous_annual_inc_joint               0.99560
continuous_dti_joint                      0.99560
continuous_mths_since_last_record         0.81010
continuous_mths_since_last_major_derog    0.69896
continuous_mths_since_last_delinq         0.47834
                                           ...   
discrete_purpose_1_one_hot                0.00000
discrete_policy_code_1_one_hot            0.00000
discrete_home_ownership_4_one_hot         0.00000
discrete_home_ownership_3_one_hot         0.00000
continuous_annual_inc                     0.00000
Length: 146, dtype: float64

In [78]:
train_new=train.sample(frac=1)

In [79]:
# train_new.to_csv(r'C:\Users\liuchang\Desktop\机器学习-王然老师\ML_dataset-作业数据集\final\train_new.csv',index=False)

In [80]:
# train_new=pd.read_csv(r'C:\Users\liuchang\Desktop\机器学习-王然老师\ML_dataset-作业数据集\final\train_new.csv',encoding='gbk')

In [81]:
import io
import multiprocessing
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict
import hyperopt.pyll
from hyperopt import fmin, tpe, hp
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

# import torch

import copy
cpu_count = 4
use_gpu = False
@dataclass
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [64])
    metric: any = hp.choice('metric', ['binary_error'])
    num_round: any = hp.choice('num_rounds', [1000])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.5, 1.0)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.8, 1.0)
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',
                                                                                   ['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt', 'dart', 'goss'])
    extra_trees: any = hp.choice('extra_tress', [False, True])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 1000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
    


In [82]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")

In [83]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LGBOpt = None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models

## 建立Baseline 
model： lightgbm  
validation：0.92018  
test：0.91796

In [84]:
fitter = LGBFitter(label='loan_status')
kfold = KFold(n_splits=5)
params = {'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': 0.019, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}

train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train_new, test, params = params)

print(params)
print(1-np.mean(error_rate))
print(accuracy_score(test['loan_status'], (test_pred>0.5).astype(np.int64)))


The minimum is attained in round 611
Finished loading model, total used 2000 iterations
The minimum is attained in round 634
Finished loading model, total used 2000 iterations
The minimum is attained in round 466
Finished loading model, total used 2000 iterations
The minimum is attained in round 650
Finished loading model, total used 2000 iterations
The minimum is attained in round 365
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.019, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.92018
0.91796


## 计算特征重要性

In [85]:
importances=models[1].feature_importance(importance_type = 'gain')
features = train_new.drop(columns='loan_status').columns.to_list()
ft=pd.DataFrame()
for i,feature in enumerate(features):
    ft.loc[i,'feature']=feature
    ft.loc[i,'importance']=importances[i]
ft.sort_values(by='importance',ascending=False)

,feature,importance
12,continuous_last_fico_range_high,509712.292458
13,continuous_last_fico_range_low,106749.348035
143,discrete_term_1_one_hot,20833.573921
3,continuous_dti,11225.863819
10,continuous_installment,9493.888779
...,...,...
68,discrete_addr_state_49_one_hot,0.000000
106,discrete_purpose_12_one_hot,0.000000
21,discrete_addr_state_2_one_hot,0.000000
67,discrete_addr_state_48_one_hot,0.000000


## drop 特征重要性为零的变量并生成新的数据集

In [86]:
del_var=ft[ft['importance']==0]
dek_var_list=del_var.feature.tolist()

train_index=train_new.shape[0]
test_index=test.shape[0]+train_index
data=pd.concat([train_new,test],axis=0,join='inner')

data.drop(dek_var_list,axis=1,inplace=True)
train_new2=data.iloc[:train_index,:]
test_new2=data.iloc[train_index:test_index,:]

## 用新的数据集再次拟合模型  
model： lightgbm  
validation：0.91932  
test：0.91804

In [87]:
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train_new2, test_new2, params = params)

print(params)
print(1-np.mean(error_rate))
print(accuracy_score(test_new2['loan_status'], (test_pred>0.5).astype(np.int64)))

The minimum is attained in round 677
Finished loading model, total used 2000 iterations
The minimum is attained in round 569
Finished loading model, total used 2000 iterations
The minimum is attained in round 807
Finished loading model, total used 2000 iterations
The minimum is attained in round 647
Finished loading model, total used 2000 iterations
The minimum is attained in round 363
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.019, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9196
0.91808


## 再次计算特征重要性 

In [88]:
importances=models[1].feature_importance(importance_type = 'gain')
features = train_new2.drop(columns='loan_status').columns.to_list()
ft=pd.DataFrame()
for i,feature in enumerate(features):
    ft.loc[i,'feature']=feature
    ft.loc[i,'importance']=importances[i]
ft.sort_values(by='importance',ascending=False)

,feature,importance
12,continuous_last_fico_range_high,545921.917114
13,continuous_last_fico_range_low,70031.115131
129,discrete_term_1_one_hot,24696.988078
3,continuous_dti,11413.972720
10,continuous_installment,9614.907197
...,...,...
58,discrete_addr_state_41_one_hot,9.298309
110,discrete_sub_grade_15_one_hot,4.223730
55,discrete_addr_state_38_one_hot,3.818060
39,discrete_addr_state_21_one_hot,3.259640


## 探索性数据分析

In [89]:
train_new2[['loan_status','continuous_annual_inc']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_annual_inc                                       \
                            count          mean            std     min   
loan_status                                                              
0                         10212.0  71983.453721   66603.868034  5360.0   
1                         39788.0  79995.170718  104286.769609     0.0   

                                                                             \
                 0%        10%      20%      30%      40%      50%      60%   
loan_status                                                                   
0            5360.0  32072.244  41000.0  49000.0  55000.0  62000.0  70000.0   
1               0.0  35000.000  43500.0  50000.0  60000.0  66500.0  75000.0   

                                                     
                 70%       80%       90%        max  
loan_status                                          
0            80000.0   94000.0  120000.0  4300012.0  
1            88322.7  104000.0  130000.0  9000000.0

In [90]:
train_new2[['loan_status','continuous_installment']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_installment                                        \
                             count        mean         std    min     0%   
loan_status                                                                
0                          10212.0  450.914086  246.087410  30.54  30.54   
1                          39788.0  423.239942  256.399089  14.77  14.77   

                                                                               \
                10%     20%     30%     40%      50%      60%     70%     80%   
loan_status                                                                     
0            170.09  249.08  298.17  342.68  396.755  472.676  538.88  645.93   
1            152.66  199.26  260.55  312.86  361.280  436.800  507.58  630.24   

                               
                 90%      max  
loan_status                    
0            799.905  1300.55  
1            786.722  1354.66

In [91]:
train_new2[['loan_status','continuous_fico_range_high']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_fico_range_high                                       \
                                 count        mean        std    min     0%   
loan_status                                                                   
0                              10212.0  690.104191  24.838219  664.0  664.0   
1                              39788.0  700.534809  32.268256  664.0  664.0   

                                                                            \
               10%    20%    30%    40%    50%    60%    70%    80%    90%   
loan_status                                                                  
0            664.0  669.0  674.0  679.0  684.0  689.0  699.0  709.0  724.0   
1            669.0  674.0  679.0  684.0  694.0  699.0  709.0  724.0  744.0   

                    
               max  
loan_status         
0            850.0  
1            850.0

In [92]:
train_new2[['loan_status','continuous_fico_range_low']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_fico_range_low                                       \
                                count        mean        std    min     0%   
loan_status                                                                  
0                             10212.0  686.104093  24.837591  660.0  660.0   
1                             39788.0  696.534634  32.267444  660.0  660.0   

                                                                            \
               10%    20%    30%    40%    50%    60%    70%    80%    90%   
loan_status                                                                  
0            660.0  665.0  670.0  675.0  680.0  685.0  695.0  705.0  720.0   
1            665.0  670.0  675.0  680.0  690.0  695.0  705.0  720.0  740.0   

                    
               max  
loan_status         
0            845.0  
1            845.0

In [93]:
train_new2[['loan_status','continuous_last_fico_range_low']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_last_fico_range_low                                    \
                                     count        mean         std  min   0%   
loan_status                                                                    
0                                  10212.0  500.803956  189.160258  0.0  0.0   
1                                  39788.0  698.884337   70.952497  0.0  0.0   

                                                                            \
               10%    20%    30%    40%    50%    60%    70%    80%    90%   
loan_status                                                                  
0              0.0  510.0  525.0  540.0  555.0  565.0  580.0  600.0  630.0   
1            625.0  660.0  675.0  690.0  705.0  715.0  730.0  750.0  775.0   

                    
               max  
loan_status         
0            830.0  
1            845.0

In [94]:
train_new2[['loan_status','continuous_last_fico_range_high']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_last_fico_range_high                                   \
                                      count        mean        std  min   0%   
loan_status                                                                    
0                                   10212.0  563.552096  50.645135  0.0  0.0   
1                                   39788.0  704.414622  60.421746  0.0  0.0   

                                                                            \
               10%    20%    30%    40%    50%    60%    70%    80%    90%   
loan_status                                                                  
0            499.0  514.0  529.0  544.0  559.0  569.0  584.0  604.0  634.0   
1            629.0  664.0  679.0  694.0  709.0  719.0  734.0  754.0  779.0   

                    
               max  
loan_status         
0            834.0  
1            850.0

In [95]:
train_new2[['loan_status','continuous_annual_inc_joint']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_annual_inc_joint                                        \
                                  count           mean           std      min   
loan_status                                                                     
0                                  57.0  116166.251228  67075.942903  44796.0   
1                                 163.0  106829.817423  46069.834366  28000.0   

                                                                              \
                  0%      10%      20%      30%      40%       50%       60%   
loan_status                                                                    
0            44796.0  65960.0  77360.8  84940.0  88200.0   99000.0  110000.0   
1            28000.0  56140.0  71400.0  78000.0  87800.0  100000.0  108400.0   

                                                     
                  70%       80%       90%       max  
loan_status                                          
0            126560.0  144000.0  165200.0  500000.0  
1            117196.8  145000.0  169546.2  340000.0

## 构造衍生特征并生成新数据集

In [96]:
train_index=train_new2.shape[0]
test_index=test_new2.shape[0]+train_index
data=pd.concat([train_new2,test],axis=0,join='inner')

In [97]:
data['continuous_pay_in_rate']=data['continuous_installment']*12/data['continuous_annual_inc']

In [98]:
data['continuous_pay_in_rate'].sort_values()

6460     0.000123
21835    0.000416
35667    0.000452
42292    0.000540
48449    0.000641
           ...   
8353     1.133040
7767     1.602305
14856    4.459100
7632          inf
16142         inf
Name: continuous_pay_in_rate, Length: 100000, dtype: float64

In [99]:
data.loc[data['continuous_pay_in_rate']==np.inf,'continuous_pay_in_rate']=999

In [100]:
data['continuous_pay_in_rate'].sort_values()

6460       0.000123
21835      0.000416
35667      0.000452
42292      0.000540
48449      0.000641
            ...    
8353       1.133040
7767       1.602305
14856      4.459100
7632     999.000000
16142    999.000000
Name: continuous_pay_in_rate, Length: 100000, dtype: float64

In [101]:
data[['loan_status','continuous_pay_in_rate']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_pay_in_rate                                          \
                             count      mean       std       min        0%   
loan_status                                                                  
0                          19986.0  0.135871  7.065984  0.000877  0.000877   
1                          80014.0  0.087207  3.531704  0.000123  0.000123   

                                                                         \
                  10%       20%       30%       40%       50%       60%   
loan_status                                                               
0            0.035353  0.048692  0.059942  0.070625  0.081403  0.093069   
1            0.026933  0.038586  0.048517  0.058117  0.067986  0.078632   

                                                  
                  70%       80%       90%    max  
loan_status                                       
0            0.106130  0.120865  0.141667  999.0  
1            0.091474  0.108048  0.132438  999.0

In [102]:
data['continuous_fico_range']=data['continuous_fico_range_high']-data['continuous_fico_range_low']
data['continuous_last_fico_range']=data['continuous_last_fico_range_high']-data['continuous_last_fico_range_low']

In [103]:
data[['loan_status','continuous_fico_range']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))

continuous_fico_range                                          \
                            count      mean       std  min   0%  10%  20%   
loan_status                                                                 
0                         19986.0  4.000050  0.007074  4.0  4.0  4.0  4.0   
1                         80014.0  4.000162  0.012745  4.0  4.0  4.0  4.0   

                                                     
             30%  40%  50%  60%  70%  80%  90%  max  
loan_status                                          
0            4.0  4.0  4.0  4.0  4.0  4.0  4.0  5.0  
1            4.0  4.0  4.0  4.0  4.0  4.0  4.0  5.0

In [104]:
data[['loan_status','continuous_last_fico_range']].groupby('loan_status').describe(percentiles=np.arange(0,1,0.1))


continuous_last_fico_range                                        \
                                 count       mean         std  min   0%  10%   
loan_status                                                                    
0                              19986.0  63.565246  161.053332  0.0  0.0  4.0   
1                              80014.0   5.552828   27.680878  0.0  0.0  4.0   

                                                              
             20%  30%  40%  50%  60%  70%  80%    90%    max  
loan_status                                                   
0            4.0  4.0  4.0  4.0  4.0  4.0  4.0  499.0  499.0  
1            4.0  4.0  4.0  4.0  4.0  4.0  4.0    4.0  499.0

In [105]:
data['continuous_inc_ratio']=data['continuous_annual_inc']/data['continuous_annual_inc_joint']

In [106]:
data['continuous_inc_ratio'].isnull().sum()

99599

In [107]:
data['continuous_annual_inc_joint'].isnull().sum()

99599

In [108]:
data['continuous_dti_ratio']=data['continuous_dti']/data['continuous_dti_joint']

In [109]:
data['continuous_dti_ratio'].isnull().sum()

99602

In [110]:
data['continuous_dti_joint'].isnull().sum()

99600

In [111]:
train_new3=data.iloc[:train_index,:]
test_new3=data.iloc[train_index:test_index,:]

In [112]:
train_new3.shape

(50000, 137)

In [113]:
test_new3.shape

(50000, 137)

## 用新数据集再次拟合模型

In [114]:
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train_new3, test_new3, params = params)

print(params)
print(1-np.mean(error_rate))
print(accuracy_score(test_new3['loan_status'], (test_pred>0.5).astype(np.int64)))

The minimum is attained in round 475
Finished loading model, total used 2000 iterations
The minimum is attained in round 474
Finished loading model, total used 2000 iterations
The minimum is attained in round 390
Finished loading model, total used 2000 iterations
The minimum is attained in round 697
Finished loading model, total used 2000 iterations
The minimum is attained in round 363
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.019, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9198
0.91786


## 再次计算特征重要性

In [115]:
importances=models[1].feature_importance(importance_type = 'gain')
features = train_new3.drop(columns='loan_status').columns.to_list()
ft=pd.DataFrame()
for i,feature in enumerate(features):
    ft.loc[i,'feature']=feature
    ft.loc[i,'importance']=importances[i]
ft.sort_values(by='importance',ascending=False)

,feature,importance
12,continuous_last_fico_range_high,497571.115600
13,continuous_last_fico_range_low,117404.444942
129,discrete_term_1_one_hot,22850.898112
3,continuous_dti,10796.317369
131,continuous_pay_in_rate,8878.254327
...,...,...
60,discrete_addr_state_44_one_hot,2.967120
124,discrete_sub_grade_29_one_hot,0.000000
45,discrete_addr_state_28_one_hot,0.000000
132,continuous_fico_range,0.000000


## drop 特征重要性为零的变量并生成新的数据集

In [116]:
del_var2=ft[ft['importance']==0]
del_var_list2=del_var2.feature.tolist()

train_new3.drop(del_var_list2,axis=1,inplace=True)
test_new3.drop(del_var_list2,axis=1,inplace=True)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## 用新的数据集拟合模型

In [117]:
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train_new3, test_new3, params = params)

print(params)
print(1-np.mean(error_rate))
print(accuracy_score(test_new3['loan_status'], (test_pred>0.5).astype(np.int64)))

The minimum is attained in round 471
Finished loading model, total used 2000 iterations
The minimum is attained in round 456
Finished loading model, total used 2000 iterations
The minimum is attained in round 431
Finished loading model, total used 2000 iterations
The minimum is attained in round 604
Finished loading model, total used 2000 iterations
The minimum is attained in round 362
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.019, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91996
0.91804


## 寻找较优参数

In [285]:
for num_leaves in [8,9,10,11,12,13,14]:
    for learning_rate in [0.015,0.016,0.017,0.018,0.019,0.020,0.021,0.022,0.023,0.024,0.025]:
        params = {'num_thread': 4, 'num_leaves':num_leaves, 'metric': 'binary', 'objective': 'binary','num_round': 2000, 'learning_rate': learning_rate, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
        train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train_new3, test_new3, params = params)
        print(params)
        print(1-np.mean(error_rate))
        print(accuracy_score(test_new3['loan_status'], (test_pred>0.5).astype(np.int64)))


The minimum is attained in round 660
Finished loading model, total used 2000 iterations
The minimum is attained in round 693
Finished loading model, total used 2000 iterations
The minimum is attained in round 639
Finished loading model, total used 2000 iterations
The minimum is attained in round 809
Finished loading model, total used 2000 iterations
The minimum is attained in round 512
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 8, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.015, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9196
0.91784
The minimum is attained in round 683
Finished loading model, total used 2000 iterations
The minimum is attained in round 671
Finished loading model, total used 2000 iterations
The minimum is attained in round 672
Finished loading model, total used 2000 iterations
The minimum is attained in round 679
Finished loading model, total used 2000 iterations
The minimum is attaine

The minimum is attained in round 509
Finished loading model, total used 2000 iterations
The minimum is attained in round 501
Finished loading model, total used 2000 iterations
The minimum is attained in round 538
Finished loading model, total used 2000 iterations
The minimum is attained in round 431
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 9, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.017, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9195599999999999
0.91814
The minimum is attained in round 573
Finished loading model, total used 2000 iterations
The minimum is attained in round 507
Finished loading model, total used 2000 iterations
The minimum is attained in round 516
Finished loading model, total used 2000 iterations
The minimum is attained in round 586
Finished loading model, total used 2000 iterations
The minimum is attained in round 433
Finished loading model, total used 2000 iterations
{'num_thre

The minimum is attained in round 431
Finished loading model, total used 2000 iterations
The minimum is attained in round 604
Finished loading model, total used 2000 iterations
The minimum is attained in round 362
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.019, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91996
0.91804
The minimum is attained in round 601
Finished loading model, total used 2000 iterations
The minimum is attained in round 466
Finished loading model, total used 2000 iterations
The minimum is attained in round 663
Finished loading model, total used 2000 iterations
The minimum is attained in round 542
Finished loading model, total used 2000 iterations
The minimum is attained in round 355
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_

The minimum is attained in round 459
Finished loading model, total used 2000 iterations
The minimum is attained in round 299
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 11, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.021, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.92002
0.918
The minimum is attained in round 407
Finished loading model, total used 2000 iterations
The minimum is attained in round 349
Finished loading model, total used 2000 iterations
The minimum is attained in round 429
Finished loading model, total used 2000 iterations
The minimum is attained in round 500
Finished loading model, total used 2000 iterations
The minimum is attained in round 299
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 11, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.022, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.91982
0.91804
The minimu

The minimum is attained in round 287
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.023, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9198999999999999
0.91814
The minimum is attained in round 389
Finished loading model, total used 2000 iterations
The minimum is attained in round 345
Finished loading model, total used 2000 iterations
The minimum is attained in round 316
Finished loading model, total used 2000 iterations
The minimum is attained in round 366
Finished loading model, total used 2000 iterations
The minimum is attained in round 268
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.024, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9198
0.9181
The minimum is attained in round 330
Finished loading model, total used 2000 iterations

{'num_thread': 4, 'num_leaves': 13, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.025, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9195800000000001
0.91832
The minimum is attained in round 542
Finished loading model, total used 2000 iterations
The minimum is attained in round 529
Finished loading model, total used 2000 iterations
The minimum is attained in round 561
Finished loading model, total used 2000 iterations
The minimum is attained in round 772
Finished loading model, total used 2000 iterations
The minimum is attained in round 399
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 14, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.015, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9195
0.91838
The minimum is attained in round 588
Finished loading model, total used 2000 iterations
The minimum is attained in round 443
Finished loading model, total used 2000 iteration

{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.019, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}  
0.91996  
0.91804  
{'num_thread': 4, 'num_leaves': 10, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.022, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}  
0.92016  
0.91794  
{'num_thread': 4, 'num_leaves': 11, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}  
0.9201400000000001  
0.91814  

In [119]:
params={'num_thread': 4, 'num_leaves': 11, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}

train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train_new3, test_new3, params = params)

print(params)
print(1-np.mean(error_rate))
print(accuracy_score(test_new3['loan_status'], (test_pred>0.5).astype(np.int64)))

The minimum is attained in round 446
Finished loading model, total used 2000 iterations
The minimum is attained in round 352
Finished loading model, total used 2000 iterations
The minimum is attained in round 430
Finished loading model, total used 2000 iterations
The minimum is attained in round 542
Finished loading model, total used 2000 iterations
The minimum is attained in round 334
Finished loading model, total used 2000 iterations
{'num_thread': 4, 'num_leaves': 11, 'metric': 'binary', 'objective': 'binary', 'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
0.9201400000000001
0.91814


## 尝试其他模型形式

In [120]:
import xgboost as xgb
import catboost as cat

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import  LogisticRegression


In [121]:
@dataclass
class XGBOpt:
    nthread: any = hp.choice('nthread', [cpu_count])
    eval_metric: any = hp.choice('eval_metric', ['error'])
    booster: any = hp.choice('booster', ['gbtree', 'dart'])
    sample_type: any = hp.choice('sample_type', ['uniform', 'weighted'])
    rate_drop: any = hp.uniform('rate_drop', 0, 0.2)
    objective: any = hp.choice('objective', ['binary:logistic'])
    max_depth: any = hp.choice('max_depth', [4, 5, 6, 7, 8])
    num_round: any = hp.choice('num_round', [100])
    eta: any = hp.uniform('eta', 0.01, 0.1)
    subsample: any = hp.uniform('subsample', 0.8, 1)
    colsample_bytree: any = hp.uniform('colsample_bytree', 0.3, 1)
    gamma: any = hp.choice('gamma', [0, 1, 5])
    min_child_weight: any = hp.uniform('min_child_weight', 0, 15)
    sampling_method: any = hp.choice('sampling_method', ['uniform', 'gradient_based'])

    @staticmethod
    def get_common_paarams():
        return {'nthread': 4, 'max_depth': 3, 'eval_metric': 'error', 'object': 'binary:logistic', 'eta': 0.01,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'num_round': 1000}


@dataclass
class CATOpt:
    thread_count: any = hp.choice('thread_count', [cpu_count])
    num_round: any = hp.choice('num_round', [100])
    objective: any = hp.choice('objective', ['CrossEntropy'])
    eval_metric: any = hp.choice('eval_metric', ['Accuracy'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    l2_leaf_reg: any = hp.uniform('l2_leaf_reg', 0, 10)  # TODO: Check range
    bootstrap_type: any = hp.choice('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS'])
    nan_mode: any = hp.choice('nan_mode', ['Forbidden', 'Min', 'Max'])
    leaf_estimation_method: any = hp.choice('leaf_estimation_method', ['Newton', 'Gradient'])
    depth: any = hp.choice('depth', [2, 3, 4, 5, 6, 7, 8])  # TODO: Check range
    max_bin: any = hp.choice('max_bin', [3, 5, 10, 15, 20, 50, 100, 500])

    @staticmethod
    def get_common_params():
        return {'thread_count': 4, 'num_round': 100, 'learning_rate': 0.01, 'objective': 'CrossEntropy',
                'eval_metric': 'Accuracy', 'depth': 3}

    
@dataclass
class RFOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [64])
    metric: any = hp.choice('metric', ['binary_error'])
    num_round: any = hp.choice('num_rounds', [1000])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.5, 1.0)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.8, 1.0)
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',
                                                                                   ['cpu'])
    boosting: any = hp.choice('boosting', ['rf'])
    extra_trees: any = hp.choice('extra_tress', [False, True])
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {"num_threads": 4, 'num_leaves': 12, 'objective': 'binary', 'metric': 'binary_error',
                'learning_rate': 0.1}

@dataclass
class LROpt:
    penalty: any = hp.choice('penalty', ['l2'])
    C: any = hp.uniform('C', 0.5, 10)
    max_iter: any = hp.choice('max_iter', [1000])
    solver: any = hp.choice('solver', ['lbfgs'])

    '''
    penalty: hyperopt.pyll.base.Apply = hp.choice('penalty', ['l2', 'none'])
    solver: hyperopt.pyll.base.Apply = hp.choice('solver', ['lbfgs'])
    C: hyperopt.pyll.base.Apply = hp.uniform('C', 0.5, 2)
    intercept_scaling: hyperopt.pyll.base.Apply = hp.uniform('intercept_scaling', 0.5, 2)
    class_weight: hyperopt.pyll.base.Apply = hp.choice('class_weight', ['balanced'])
    max_iter: hyperopt.pyll.base.Apply = hp.choice('max_iter', [10000])
    n_jobs: hyperopt.pyll.base.Apply = hp.choice('n_jobs', [4])
    '''

    @staticmethod
    def get_common_params():
        return {'penalty': 'l2', 'C': 0.5, 'solver': 'lbfgs'}



In [122]:
class XGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: XGBOpt = None, max_eval=100):
        super(XGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = XGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = xgb.DMatrix(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = xgb.DMatrix(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [(dtrain, 'train'), (deval, 'eval')]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = xgb.train(use_params, dtrain, num_round, evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = xgb.train(use_params, dtrain, num_round, evallist)
                output = buf.getvalue().split("\n")
                self.best_round = num_round
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(xgb.DMatrix(eval_df.drop(columns=[self.label])), ntree_limit=self.best_round)
            else:
                y_pred = (self.clf.predict(xgb.DMatrix(eval_df.drop(columns=[self.label])),
                                           ntree_limit=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(xgb.DMatrix(eval_df.drop(columns=[self.label])),
                                              ntree_limit=self.best_round)
                else:
                    y_pred = (self.clf.predict(xgb.DMatrix(eval_df.drop(columns=[self.label])),
                                               ntree_limit=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        result_models = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            result_models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(xgb.DMatrix(eval_df.drop(columns=self.label)),
                                                   ntree_limit=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(xgb.DMatrix(eval_df.drop(columns=[self.label])),
                                          ntree_limit=self.best_round)
            else:
                y_pred = (self.clf.predict(xgb.DMatrix(eval_df.drop(columns=[self.label])),
                                           ntree_limit=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(xgb.DMatrix(dtest), ntree_limit=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, result_models
    

class CATFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: CATOpt = None, max_eval=100):
        super(CATFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = CATOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = cat.Pool(data=train_df.drop(columns=[self.label]), label=train_df[self.label])
        deval = cat.Pool(data=eval_df.drop(columns=[self.label]), label=eval_df[self.label])
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)
        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = cat.train(params=use_params,
                                     pool=dtrain,
                                     evals=deval,
                                     num_boost_round=num_round
                                     )

                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0

            for idx in range(1, num_round + 1):
                if len(output[idx].split("\t")) == 3:
                    temp = 1 - float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = cat.train(params=use_params,
                                     pool=dtrain,
                                     evals=deval,
                                     num_boost_round=num_round
                                     )
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]),
                                          ntree_end=self.best_round - 1)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           ntree_end=self.best_round - 1) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]),
                                              ntree_end=self.best_round - 1)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),

                                               ntree_end=self.best_round - 1) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data
        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label),
                                                   ntree_end=self.best_round - 1)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), ntree_end=self.best_round - 1)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           ntree_end=self.best_round - 1) > 0.5).astype(
                    int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest,
                                          ntree_end=self.best_round - 1)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result


class RFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: RFOpt = None, max_eval=100):
        super(RFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = RFOpt()
        self.clf = None

    def train(self, train_df, eval_df, params=None):
        x_train, y_train, x_eval, y_eval = train_df.drop(columns=[self.label]), train_df[self.label], \
                                           eval_df.drop(columns=[self.label]), eval_df[self.label],
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)
        self.clf = SVC(**use_params)
        self.clf.fit(X=x_train, y=y_train)
        preds = self.clf.predict(X=x_eval)
        output = self.get_loss(y_pred=preds, y=y_eval)

        return output

    def search(self, train_df, eval_df):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]))
            else:
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label])).astype(int)

            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]))
                else:
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label])).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data
        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.iloc[train_id, :]
            eval_df = train_data.iloc[eval_id, :]
            self.train(train_df, eval_df, params)
            models.append(deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict_proba(eval_df.drop(columns=self.label))[:, 1]
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params)
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label))
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]))
            else:
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label])).astype(int)

            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict_proba(dtest)[:, 1]
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models
    

class LRFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LROpt = None, max_eval=100):
        super(LRFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LROpt()
        self.clf = None

    def train(self, train_df, eval_df, params=None):
        x_train, y_train, x_eval, y_eval = train_df.drop(columns=[self.label]), train_df[self.label], \
                                           eval_df.drop(columns=[self.label]), eval_df[self.label],

        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)
        self.clf = LogisticRegression(**use_params)
        self.clf.fit(X=x_train, y=y_train)
        preds = self.clf.predict(X=x_eval)
        output = self.get_loss(y_pred=preds, y=y_eval)
        return output

    def search(self, train_df, eval_df):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]))
            else:
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label])).astype(int)

            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.iloc[train_id, :]
                eval_df = data.iloc[eval_id, :]
                self.train(train_df, eval_df, params)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]))
                else:
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label])).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.iloc[train_id]
            eval_df = train_data.iloc[eval_id]
            self.train(train_df, eval_df, params)
            models.append(deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label))
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]))
            else:
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label])).astype(int)

            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models


## 拟合xgboost

In [123]:
fitter_xgb = XGBFitter(label='loan_status')


In [294]:
params={'num_thread': 4, 'max_depth': 3, 'objective': 'binary:logistic', 'num_round': 2000, 'eta': 0.02, 'subsample': 0.8}

train_pred_xgb, test_pred_xgb, error_rate_xgb, models_xgb =fitter_xgb.train_k_fold(kfold, train_new3, test_new3, params = params)

print(params)
print(1-np.mean(error_rate_xgb))
print(accuracy_score(test_new3['loan_status'], (test_pred_xgb>0.5).astype(np.int64)))

The minimum is attained in round 799


D:\Anaconda\lib\site-packages\xgboost\core.py:101: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


The minimum is attained in round 598
The minimum is attained in round 867
The minimum is attained in round 840
The minimum is attained in round 633
{'num_thread': 4, 'max_depth': 3, 'objective': 'binary:logistic', 'num_round': 2000, 'eta': 0.02, 'subsample': 0.8}
0.91974
0.91826


## 寻找较优参数

In [302]:
for max_depth in [2,3,4,5,6]:
    for eta in [0.015,0.016,0.017,0.018,0.019,0.020,0.021,0.022,0.023,0.024,0.025]:
        params={'num_thread': 4, 'max_depth': max_depth, 'objective': 'binary:logistic', 'num_round': 2000, 'eta':eta, 'subsample': 0.8}
        train_pred_xgb, test_pred_xgb, error_rate_xgb, models_xgb =fitter_xgb.train_k_fold(kfold, train_new3, test_new3, params = params)
        print(params)
        print(1-np.mean(error_rate_xgb))
        print(accuracy_score(test_new3['loan_status'], (test_pred_xgb>0.5).astype(np.int64)))


The minimum is attained in round 1727
The minimum is attained in round 1142
The minimum is attained in round 1262
The minimum is attained in round 1494
The minimum is attained in round 1506
{'num_thread': 4, 'max_depth': 2, 'objective': 'binary:logistic', 'num_round': 2000, 'eta': 0.015, 'subsample': 0.8}
0.92002
0.90106
The minimum is attained in round 1725
The minimum is attained in round 1141
The minimum is attained in round 1223
The minimum is attained in round 1661
The minimum is attained in round 1254
{'num_thread': 4, 'max_depth': 2, 'objective': 'binary:logistic', 'num_round': 2000, 'eta': 0.016, 'subsample': 0.8}
0.9199999999999999
0.9188
The minimum is attained in round 1299
The minimum is attained in round 960
The minimum is attained in round 1176
The minimum is attained in round 1661
The minimum is attained in round 1254
{'num_thread': 4, 'max_depth': 2, 'objective': 'binary:logistic', 'num_round': 2000, 'eta': 0.017, 'subsample': 0.8}
0.92002
0.91898
The minimum is attaine

KeyboardInterrupt: 

In [124]:
params={'num_thread': 4, 'max_depth': 2, 'objective': 'binary:logistic', 'num_round': 2000, 'eta': 0.018, 'subsample': 0.8}

train_pred_xgb, test_pred_xgb, error_rate_xgb, models_xgb =fitter_xgb.train_k_fold(kfold, train_new3, test_new3, params = params)

print(params)
print(1-np.mean(error_rate_xgb))
print(accuracy_score(test_new3['loan_status'], (test_pred_xgb>0.5).astype(np.int64)))

The minimum is attained in round 1229


D:\Anaconda\lib\site-packages\xgboost\core.py:101: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


The minimum is attained in round 958
The minimum is attained in round 1176
The minimum is attained in round 1497
The minimum is attained in round 1085
{'num_thread': 4, 'max_depth': 2, 'objective': 'binary:logistic', 'num_round': 2000, 'eta': 0.018, 'subsample': 0.8}
0.92016
0.919


## 拟合catboost

In [133]:
train_new4=train_new3.drop([x for x in train_new3.columns.to_list() if x.startswith('discrete')],axis=1)
test_new4=test_new3.drop([x for x in test_new3.columns.to_list() if x.startswith('discrete')],axis=1)

test_new3['loan_status']

0        1
1        1
2        1
3        1
4        1
        ..
49995    1
49996    0
49997    1
49998    1
49999    1
Name: loan_status, Length: 50000, dtype: int64

In [176]:
import catboost
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

cat_features_index = [0,1,2,3,4,5,6]

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [20],
         'silent': [True]}
cb = catboost.CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
cb_model.fit(train_new3.drop('loan_status',axis=1), train_new3['loan_status'])

# With Categorical features
clf = catboost.CatBoostClassifier(eval_metric="AUC", depth=3, silent=True,
                                  iterations= 20, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train_new3.drop('loan_status',axis=1),train_new3['loan_status'])
print(auc(clf, train_new3.drop('loan_status',axis=1), test_new3.drop('loan_status',axis=1)))

train_pred_cat=cb_model.predict_proba(train_new3.drop('loan_status',axis=1))[:,1]
test_pred_cat=cb_model.predict_proba(test_new3.drop('loan_status',axis=1))[:,1]

print(accuracy_score(train_new3['loan_status'], (train_pred_cat>0.5).astype(np.int64)))
print(accuracy_score(test_new3['loan_status'], (test_pred_cat>0.5).astype(np.int64)))


0.92096
0.91762


## 合并三种预测结果作为stacking特征

In [158]:
train_ft=pd.DataFrame({'train_ft_lgb':train_pred,'train_ft_xgb':train_pred_xgb,'train_ft_cat':train_pred_cat,'loan_status':train_new3['loan_status']})
test_ft=pd.DataFrame({'test_ft_lgb':test_pred,'test_ft_xgb':test_pred_xgb,'test_ft_cat':test_pred_cat,'loan_status':test_new3['loan_status']})

## 拟合逻辑回归

In [190]:
fitter = LRFitter(label='loan_status')
kfold = KFold(n_splits=5)
for C in np.arange(0.5,2,0.1): 
    for max_iter in np.arange(1000,2000,100):
        params = {'C':C,'max_iter':max_iter }
        train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train_ft, test_ft, params = params)
        print(params)
        print(1-np.mean(error_rate))
        print(accuracy_score(test_ft['loan_status'], (test_pred>0.5).astype(np.int64)))


{'C': 0.5, 'max_iter': 1000}
0.92084
0.91734
{'C': 0.5, 'max_iter': 1100}
0.92084
0.91734
{'C': 0.5, 'max_iter': 1200}
0.92084
0.91734
{'C': 0.5, 'max_iter': 1300}
0.92084
0.91734
{'C': 0.5, 'max_iter': 1400}
0.92084
0.91734
{'C': 0.5, 'max_iter': 1500}
0.92084
0.91734
{'C': 0.5, 'max_iter': 1600}
0.92084
0.91734
{'C': 0.5, 'max_iter': 1700}
0.92084
0.91734
{'C': 0.5, 'max_iter': 1800}
0.92084
0.91734
{'C': 0.5, 'max_iter': 1900}
0.92084
0.91734
{'C': 0.6, 'max_iter': 1000}
0.92086
0.91738
{'C': 0.6, 'max_iter': 1100}
0.92086
0.91738
{'C': 0.6, 'max_iter': 1200}
0.92086
0.91738
{'C': 0.6, 'max_iter': 1300}
0.92086
0.91738
{'C': 0.6, 'max_iter': 1400}
0.92086
0.91738
{'C': 0.6, 'max_iter': 1500}
0.92086
0.91738
{'C': 0.6, 'max_iter': 1600}
0.92086
0.91738
{'C': 0.6, 'max_iter': 1700}
0.92086
0.91738
{'C': 0.6, 'max_iter': 1800}
0.92086
0.91738
{'C': 0.6, 'max_iter': 1900}
0.92086
0.91738
{'C': 0.7, 'max_iter': 1000}
0.92086
0.91736
{'C': 0.7, 'max_iter': 1100}
0.92086
0.91736
{'C': 0.7,

{'C': 1.8999999999999997, 'max_iter': 1700}
0.92082
0.91738
{'C': 1.8999999999999997, 'max_iter': 1800}
0.92082
0.91738
{'C': 1.8999999999999997, 'max_iter': 1900}
0.92082
0.91738


In [170]:
params = {'C':0.6 }
train_pred, test_pred, error_rate, models =fitter.train_k_fold(kfold, train_ft, test_ft, params = params)
print(params)
print(1-np.mean(error_rate))
print(accuracy_score(test_ft['loan_status'], (test_pred>0.5).astype(np.int64)))

{'C': 0.6}
0.92086
0.91738


In [171]:
y_test=(test_pred>0.5).astype(np.int64)

## 比较三个模型和融合模型效果

In [180]:
from sklearn.ensemble import GradientBoostingClassifier

In [186]:
acc=pd.DataFrame({'model':['lgb','xgb','cat','stacking'],'val':[0.92014,0.92016,0.92096,0.92086],'test':[0.91814,0.919,0.91762,0.91738]})

In [187]:
acc

,model,val,test
0,lgb,0.92014,0.91814
1,xgb,0.92016,0.91900
2,cat,0.92096,0.91762
3,stacking,0.92086,0.91738
